In [ ]:
import requests

from math import log
from statistics import mean

# import numpy as np
import pandas as pd

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import gensim

### Import and Preprocess Data

Kaggle Dataset:
https://www.kaggle.com/datasets/abisheksudarshan/topic-modeling-for-research-articles?resource=download

In [ ]:
df = pd.read_csv("dataset/Test.csv")

In [ ]:
nltk_sw = nltk.corpus.stopwords.words('english')

def get_wordnet_tag(tag):
    tag_map = {
        "J": nltk.corpus.wordnet.ADJ,
        "N": nltk.corpus.wordnet.NOUN,
        "V": nltk.corpus.wordnet.VERB,
        "R": nltk.corpus.wordnet.ADV
    }
    return tag_map.get(tag[0].upper(), nltk.corpus.wordnet.NOUN)

def get_tokens(text):
    tokens = nltk.RegexpTokenizer("[\w']+").tokenize(text)
    tokens = nltk.pos_tag(tokens)
    tokens = [nltk.stem.WordNetLemmatizer().lemmatize(word, get_wordnet_tag(tag)) for word, tag in tokens]
    # tokens = [word for word in tokens if word not in nltk_sw]
    return tokens

def get_corpus(docs):
    return docs.apply(get_tokens)

# docs must be a list of lists of words
def get_stopwords(corpus, tfidf=False, threshold=0.5):
    words = {}
    for i, doc in enumerate(corpus):
        for word in doc:
            words[word] = words.get(word, {})
            words[word][i] = (words[word].get(i, 0) + 1) if tfidf else 1

    for word in words:
        if tfidf:
            tf = sum(words[word].values())
            df = len(words[word].values())
            tfidf = tf / df
            words[word] = tfidf
        else:
            words[word] = len(words[word].values()) / len(corpus)
    
    s = pd.Series(words)
    return s[s > threshold].sort_values()

def remove_corpus_stopwords(corpus):
    sw = get_stopwords(corpus)
    return corpus.apply(lambda tokens: [word for word in tokens if word not in sw])

In [ ]:
docs = df["ABSTRACT"]
corpus = get_corpus(docs)

In [ ]:
sw = get_stopwords(corpus, threshold=0)
sw[-50:]

In [ ]:
corpus = remove_corpus_stopwords(corpus)
dictionary = gensim.corpora.Dictionary(corpus)
word_freq = [dictionary.doc2bow(word) for word in corpus]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=word_freq,
                                            id2word=dictionary,
                                            num_topics=4,
                                            random_state=100,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto",
                                            per_word_topics=True)

[print(row[0], '\n', row[1], '\n') for row in lda_model.print_topics(num_words=20)]